In [1]:
import correctmatch
correctmatch.precompile()

import numpy as np
import pandas as pd

# Tests with a basic dataset

In [2]:
arr = np.random.randint(1, 5, size=(1000, 5))
arr[:5, :]

array([[1, 1, 1, 3, 1],
       [3, 3, 2, 3, 3],
       [3, 3, 4, 3, 2],
       [3, 4, 3, 3, 3],
       [3, 4, 2, 2, 4]])

In [3]:
correctmatch.uniqueness(arr)

0.38

In [4]:
fitted_model = correctmatch.fit_model(arr)

In [35]:
fitted_model

<PyCall.jlwrap correctmatch.Copula.GaussianCopula(
Σ: PDMats.PDMat{Float64,Array{Float64,2}}(5, [1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0], Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
[1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0])
marginals: Distributions.Distribution{Distributions.Univariate,Distributions.Discrete}[Distributions.Categorical{Float64}(K=4, p=[0.266, 0.247, 0.244, 0.243]), Distributions.Categorical{Float64}(K=4, p=[0.287, 0.272, 0.226, 0.215]), Distributions.Categorical{Float64}(K=4, p=[0.264, 0.249, 0.245, 0.242]), Distributions.Categorical{Float64}(K=4, p=[0.266, 0.252, 0.249, 0.233]), Distributions.Categorical{Float64}(K=4, p=[0.272, 0.254, 0.245, 0.229])]
)
>

In [7]:
# Covariance matrix
fitted_model.Σ.mat.round(2)

array([[ 1.  ,  0.  ,  0.06,  0.13,  0.  ],
       [ 0.  ,  1.  ,  0.06, -0.  ,  0.1 ],
       [ 0.06,  0.06,  1.  ,  0.09,  0.  ],
       [ 0.13, -0.  ,  0.09,  1.  ,  0.  ],
       [ 0.  ,  0.1 ,  0.  ,  0.  ,  1.  ]])

In [8]:
# Fitted marginals
fitted_model.marginals

[<PyCall.jlwrap Distributions.Categorical{Float64}(K=4, p=[0.277, 0.245, 0.245, 0.233])>,
 <PyCall.jlwrap Distributions.Categorical{Float64}(K=4, p=[0.263, 0.262, 0.239, 0.236])>,
 <PyCall.jlwrap Distributions.Categorical{Float64}(K=4, p=[0.27, 0.26, 0.239, 0.231])>,
 <PyCall.jlwrap Distributions.Categorical{Float64}(K=4, p=[0.272, 0.245, 0.242, 0.241])>,
 <PyCall.jlwrap Distributions.Categorical{Float64}(K=4, p=[0.257, 0.257, 0.244, 0.242])>]

In [9]:
fitted_arr = correctmatch.sample_model(fitted_model, 1000)
fitted_arr[1:5, :]

array([[4, 2, 1, 4, 1],
       [4, 2, 3, 2, 3],
       [1, 3, 1, 3, 1],
       [1, 3, 3, 1, 4]], dtype=int64)

In [39]:
correctmatch.uniqueness(fitted_arr)

0.393

# Tests with a real dataset

In [13]:
adults = pd.read_csv('adults.csv')
adults.head()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,7,13,4,1,1,4,1,40,39,0
1,50,6,13,2,4,0,4,1,13,39,0
2,38,4,9,0,6,1,4,1,40,39,0
3,53,4,7,2,6,0,2,1,40,39,0
4,28,4,13,2,10,5,2,0,40,5,0


In [14]:
len(adults)

32561

In [11]:
adults.values[1:5, :]

array([[50,  6, 13,  2,  4,  0,  4,  1, 13, 39,  0],
       [38,  4,  9,  0,  6,  1,  4,  1, 40, 39,  0],
       [53,  4,  7,  2,  6,  0,  2,  1, 40, 39,  0],
       [28,  4, 13,  2, 10,  5,  2,  0, 40,  5,  0]])

In [42]:
correctmatch.uniqueness(adults.values)

0.793894536408587

In [15]:
fitted_model = correctmatch.fit_model(adults.values)

In [16]:
fitted_adults = correctmatch.sample_model(fitted_model, adults.shape[0])
fitted_adults[1:5, :]

array([[34,  2,  2,  2,  5,  2,  1,  1,  1,  1,  1],
       [36,  1,  2,  5,  5,  5,  1,  2, 18,  1,  2],
       [25,  1,  5,  1,  1,  1,  1,  1,  0,  1,  1],
       [22,  1,  2,  1,  4,  1,  1,  1,  3,  1,  1]], dtype=int64)

In [17]:
correctmatch.uniqueness(fitted_adults)

0.735388962255459

# Tests with a sample

In [18]:
adults.shape[0]

32561

In [19]:
fitted_model = correctmatch.fit_model(adults.sample(3000).values)

In [20]:
fitted_adults = correctmatch.sample_model(fitted_model, adults.shape[0])
fitted_adults[1:5, :]

array([[15,  1,  1,  2,  3,  2,  0,  1,  6,  0,  2],
       [87,  6,  7,  3, 11,  4,  0,  1,  8,  0,  1],
       [20,  5,  2,  3,  7,  5,  0,  2, 12,  0,  1],
       [25,  1,  7,  4,  6,  5,  2,  2,  0, 18,  2]], dtype=int64)

In [21]:
correctmatch.uniqueness(fitted_adults)

0.7317035717576241

In [28]:
fitted_model = correctmatch.fit_model(adults.sample(300).values)

In [31]:
fitted_adults = correctmatch.sample_model(fitted_model, adults.shape[0])
fitted_adults[1:5, :]

array([[ 1,  1,  0,  0,  3,  1,  1,  1,  1,  1,  0],
       [ 4,  1,  0,  0,  0,  0,  1,  2,  9,  1,  0],
       [ 3,  1,  1,  0,  1,  0,  1,  1,  0,  1,  0],
       [15,  1,  1,  2, 11,  3,  1,  1,  2,  1,  1]], dtype=int64)

In [32]:
correctmatch.uniqueness(fitted_adults)

0.8036608212278493

In [33]:
fitted_model.marginals

[<PyCall.jlwrap Distributions.Geometric{Float64}(p=0.10869565198824072)>,
 <PyCall.jlwrap correctmatch.Marginal.Logarithmic{Float64}(p=0.48712140086711714)>,
 <PyCall.jlwrap Distributions.Geometric{Float64}(p=0.28037382749004985)>,
 <PyCall.jlwrap Distributions.NegativeBinomial{Float64}(r=15.6409920341873, p=0.9455897202758063)>,
 <PyCall.jlwrap Distributions.Geometric{Float64}(p=0.2564102608927633)>,
 <PyCall.jlwrap Distributions.NegativeBinomial{Float64}(r=1112.2036836687237, p=0.9989521189965755)>,
 <PyCall.jlwrap correctmatch.Marginal.Logarithmic{Float64}(p=0.34720426209046445)>,
 <PyCall.jlwrap Distributions.Categorical{Float64}(K=2, p=[0.633333, 0.366667])>,
 <PyCall.jlwrap Distributions.Geometric{Float64}(p=0.2189780988533817)>,
 <PyCall.jlwrap correctmatch.Marginal.Logarithmic{Float64}(p=0.3140494205442313)>,
 <PyCall.jlwrap Distributions.Geometric{Float64}(p=0.8333333359198016)>]